In [1]:
# cd /home/ubuntu/FedEM/

In [2]:
# Import General Libraries
import os
import argparse
import torch
import copy
import pickle
import random
import numpy as np
import pandas as pd

# Import FedEM based Libraries
from utils.utils import *
from utils.constants import *
from utils.args import *
from run_experiment import *
from models import *

# Import Transfer Attack
from transfer_attacks.Personalized_NN import *
from transfer_attacks.Params import *
from transfer_attacks.Transferer import *
from transfer_attacks.Args import *
from transfer_attacks.TA_utils import *
from transfer_attacks.Boundary_Transferer import *

In [3]:
# Generating Empty Aggregator to be loaded 

setting = 'FedAvg'

if setting == 'FedEM':
    nL = 3
else:
    nL = 1

# Manually set argument parameters
args_ = Args()
args_.experiment = "cifar100"
args_.method = setting
args_.decentralized = False
args_.sampling_rate = 1.0
args_.input_dimension = None
args_.output_dimension = None
args_.n_learners= nL
args_.n_rounds = 10
args_.bz = 128
args_.local_steps = 1
args_.lr_lambda = 0
args_.lr =0.03
args_.lr_scheduler = 'multi_step'
args_.log_freq = 10
args_.device = 'cuda'
args_.optimizer = 'sgd'
args_.mu = 0
args_.communication_probability = 0.1
args_.q = 1
args_.locally_tune_clients = False
args_.seed = 1234
args_.verbose = 1
args_.save_path = 'weights/cifar100/fedAvg/'
args_.validation = False

# Generate the dummy values here
aggregator, clients = dummy_aggregator(args_, num_user=40)

==> Clients initialization..
===> Building data iterators..


100%|█████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 406.45it/s]


===> Initializing clients..


100%|██████████████████████████████████████████████████████████████| 100/100 [00:13<00:00,  7.53it/s]


==> Test Clients initialization..
===> Building data iterators..


0it [00:00, ?it/s]


===> Initializing clients..


0it [00:00, ?it/s]


++++++++++++++++++++++++++++++
Global..
Train Loss: 4.613 | Train Acc: 1.193% |Test Loss: 4.615 | Test Acc: 1.000% |
++++++++++++++++++++++++++++++++++++++++++++++++++
################################################################################


In [4]:
# Compiling Dataset from Clients
# Combine Validation Data across all clients as test
data_x = []
data_y = []

for i in range(len(clients)):
    daniloader = clients[i].test_iterator
    for (x,y,idx) in daniloader.dataset:
        data_x.append(x)
        data_y.append(y)

data_x = torch.stack(data_x)
try:
    data_y = torch.stack(data_y)        
except:
    data_y = torch.FloatTensor(data_y) 
    
dataloader = Custom_Dataloader(data_x, data_y)

In [5]:
type(clients[0])

client.Client

In [6]:
# Import Model Weights
num_models = 40

np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

if setting == 'local':

#     args_.save_path = 'weights/final/femnist/fig1_take3/local_benign/'
#     args_.save_path ='/home/ubuntu/code/FedEM/weights/cifar100/benign'
    aggregator.load_state(args_.save_path)
    
    model_weights = []
#     weights = np.load("weights/final/femnist/fig1_take3/local_benign/train_client_weights.npy")
#     weights = np.load('weights/final/femnist/fig1_take3/local_adv/train_client_weights.npy')
    weights = np.load(args_.save_path+'/train_client_weights.npy')
    
    for i in range(num_models):
        model_weights += [weights[i]]

    # Generate the weights to test on as linear combinations of the model_weights
    models_test = []
    
    for i in range(num_models):
        new_model = copy.deepcopy(aggregator.clients[i].learners_ensemble.learners[0].model)
        new_model.eval()
        models_test += [new_model]

elif setting == 'FedAvg':
    
#     args_.save_path = 'weights/final/femnist/fig1_take3/fedavg_benign/'
#     args_.save_path = 'weights/final/femnist/fig1_take3/FedAvg_adv/'
    aggregator.load_state(args_.save_path)
    
    # This is where the models are stored -- one for each mixture --> learner.model for nn
    hypotheses = aggregator.global_learners_ensemble.learners

    # obtain the state dict for each of the weights 
    weights_h = []

    for h in hypotheses:
        weights_h += [h.model.state_dict()]

#     weights = np.load("weights/final/femnist/fig1_take3/fedavg_benign/train_client_weights.npy")
    weights = np.load(args_.save_path+'/train_client_weights.npy')
    
    # Set model weights
    model_weights = []

    for i in range(num_models):
        model_weights += [weights[i]]

    # Generate the weights to test on as linear combinations of the model_weights
    models_test = []

    for (w0) in model_weights:
        # first make the model with empty weights
        new_model = copy.deepcopy(hypotheses[0].model)
        new_model.eval()
        new_weight_dict = copy.deepcopy(weights_h[0])
        for key in weights_h[0]:
            new_weight_dict[key] = w0[0]*weights_h[0][key] 
        new_model.load_state_dict(new_weight_dict)
        models_test += [new_model]

elif setting == 'FedEM':
    
#     args_.save_path = 'weights/final/femnist/fig1_take3/fedem_benign/'
#     args_.save_path = 'weights/final/femnist/fig1_take3/fedem_adv/'
#     args_.save_path = 'weights/final/femnist/figperturb/fedem_avg_p0_1/'
    aggregator.load_state(args_.save_path)
    
    # This is where the models are stored -- one for each mixture --> learner.model for nn
    hypotheses = aggregator.global_learners_ensemble.learners
    print(len(hypotheses))

    # obtain the state dict for each of the weights 
    weights_h = []

    for h in hypotheses:
        weights_h += [h.model.state_dict()]

#     weights = np.load("weights/final/femnist/fig1_take3/fedem_benign/train_client_weights.npy")
#     weights = np.load("weights/final/femnist/fig1_take3/fedem_adv/train_client_weights.npy")
    weights = np.load(args_.save_path+"/train_client_weights.npy")

    # Set model weights
    model_weights = []

    for i in range(num_models):
        model_weights += [weights[i]]


    # Generate the weights to test on as linear combinations of the model_weights
    models_test = []

    for wts in model_weights:
        # first make the model with empty weights
        new_model = copy.deepcopy(hypotheses[0].model)
        new_model.eval()
        new_weight_dict = copy.deepcopy(weights_h[0])
        for key in weights_h[0]:
            new_weight_dict[key] = 0.0
            for i in range(0, len(wts)):
#                 print(wts[i])
#                 print(type(weights_h[i][key]))
#                 print(type(wts[i]*weights_h[i][key]))
                new_weight_dict[key] += wts[i]*weights_h[i][key]
        new_model.load_state_dict(new_weight_dict)
        models_test += [new_model]
    
    test_models = []
    for wts in model_weights:
        test_models.append((wts, hypotheses))
        
    print(len(models_test))

In [7]:
nw = models_test[0].eval()
a,b,c = dataloader.select_single()

print(b.size())
models_test[0].eval()
nw(x=b).size()

torch.Size([1, 3, 32, 32])


torch.Size([1, 100])

In [8]:
# Here we will make a dictionary that will hold results
logs_adv = []

for i in range(num_models):
    adv_dict = {}
    adv_dict['orig_acc_transfers'] = None
    adv_dict['orig_similarities'] = None
    adv_dict['adv_acc_transfers'] = None
    adv_dict['adv_similarities_target'] = None
    adv_dict['adv_similarities_untarget'] = None
    adv_dict['adv_target'] = None
    adv_dict['adv_miss'] = None
    adv_dict['metric_alignment'] = None
    adv_dict['ib_distance_legit'] = None
    adv_dict['ib_distance_adv'] = None

    logs_adv += [adv_dict]

In [9]:
# Perform transfer attack from one client to another and record stats

# Run Measurements for both targetted and untargeted analysis
new_num_models = len(models_test)
victim_idxs = range(new_num_models)
custom_batch_size = 500
eps = 4.5


for adv_idx in victim_idxs:
    print("\t Adv idx:", adv_idx)
    
    # loading the data of a particular client
    dataloader = load_client_data(clients = clients, conditional=False, c_id = adv_idx, mode = 'test', models = [models_test[adv_idx]]) # or test/train
    
    print(dataloader.x_data.size())
    print(dataloader.y_data.size())
    
    batch_size = min(custom_batch_size, dataloader.y_data.shape[0])
#     print("dataset size", len(dataloader.y_data))
    
    t1 = Transferer(models_list=models_test, dataloader=dataloader) # models_test: models to be evaluated, returning data for the paritcular client
    t1.generate_victims(victim_idxs) # loading the models of all the victims
    
    # Perform Targeted PGD Attacks
    t1.atk_params = PGD_Params()
    t1.atk_params.set_params(batch_size=batch_size, iteration = 10,
                   target = 3, x_val_min = torch.min(data_x), x_val_max = torch.max(data_x),
                   step_size = 0.01, step_norm = "inf", eps = eps, eps_norm = 2)
    
    
    # generate adv using the current model and attack the reamining using the generated examples.
    t1.generate_advNN(adv_idx) # 
    t1.generate_xadv(atk_type = "pgd")
#     print(test_models[adv_idx][0])
#     print(test_models[adv_idx][1][0])
    t1.send_to_victims(victim_idxs, conditional=False) # attacking all the clients using the given model

    # Log Performance
    logs_adv[adv_idx]['orig_acc_transfers'] = copy.deepcopy(t1.orig_acc_transfers)
    logs_adv[adv_idx]['orig_similarities'] = copy.deepcopy(t1.orig_similarities)
    logs_adv[adv_idx]['adv_acc_transfers'] = copy.deepcopy(t1.adv_acc_transfers)
    logs_adv[adv_idx]['adv_similarities_target'] = copy.deepcopy(t1.adv_similarities)        
    logs_adv[adv_idx]['adv_target'] = copy.deepcopy(t1.adv_target_hit)

    # Miss attack
    t1.atk_params.set_params(batch_size=batch_size, iteration = 10,
                   target = -1, x_val_min = torch.min(data_x), x_val_max = torch.max(data_x),
                   step_size = 0.01, step_norm = "inf", eps = eps, eps_norm = 2)
    t1.generate_xadv(atk_type = "pgd")
    t1.send_to_victims(victim_idxs, conditional=False)
    logs_adv[adv_idx]['adv_miss'] = copy.deepcopy(t1.adv_acc_transfers) # adv_miss -> acc of client being attacked on adv examples
    logs_adv[adv_idx]['adv_similarities_untarget'] = copy.deepcopy(t1.adv_similarities)
    

	 Adv idx: 0
120
torch.Size([120, 3, 32, 32])
torch.Size([120])
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.2333, device='cuda:0')
self.adv_test_acc tensor(0.1333, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.2333, device='cuda:0')
self.adv_test_acc tensor(0.1333, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.2333, device='cuda:0')
self.adv_test_acc tensor(0.1333, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.2333, device='cuda:0')
self.adv_test_acc tensor(0.1333, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.2333, device='cuda:0')
self.adv_test_acc tensor(0.1333, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.2333, device='cuda:0')
self.adv_test_acc tensor(0.1333, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.2333, device='cuda:0')
self.adv_test_

x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.2333, device='cuda:0')
self.adv_test_acc tensor(0.0083, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.2333, device='cuda:0')
self.adv_test_acc tensor(0.0083, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.2333, device='cuda:0')
self.adv_test_acc tensor(0.0083, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.2333, device='cuda:0')
self.adv_test_acc tensor(0.0083, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.2333, device='cuda:0')
self.adv_test_acc tensor(0.0083, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.2333, device='cuda:0')
self.adv_test_acc tensor(0.0083, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.2333, device='cuda:0')
self.adv_test_acc tensor(0.0083, device='cuda:0')
batch_size 120
x_adv torch.S

x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.2250, device='cuda:0')
self.adv_test_acc tensor(0., device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.2250, device='cuda:0')
self.adv_test_acc tensor(0., device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.2250, device='cuda:0')
self.adv_test_acc tensor(0., device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.2250, device='cuda:0')
self.adv_test_acc tensor(0., device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.2250, device='cuda:0')
self.adv_test_acc tensor(0., device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.2250, device='cuda:0')
self.adv_test_acc tensor(0., device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.2250, device='cuda:0')
self.adv_test_acc tensor(0., device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_a

x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.2167, device='cuda:0')
self.adv_test_acc tensor(0.1000, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.2167, device='cuda:0')
self.adv_test_acc tensor(0.1000, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.2167, device='cuda:0')
self.adv_test_acc tensor(0.1000, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.2167, device='cuda:0')
self.adv_test_acc tensor(0.1000, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.2167, device='cuda:0')
self.adv_test_acc tensor(0.1000, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.2167, device='cuda:0')
self.adv_test_acc tensor(0.1000, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.2167, device='cuda:0')
self.adv_test_acc tensor(0.1000, device='cuda:0')
batch_size 120
x_adv torch.S

x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.3000, device='cuda:0')
self.adv_test_acc tensor(0.2083, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.3000, device='cuda:0')
self.adv_test_acc tensor(0.2083, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.3000, device='cuda:0')
self.adv_test_acc tensor(0.2083, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.3000, device='cuda:0')
self.adv_test_acc tensor(0.2083, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.3000, device='cuda:0')
self.adv_test_acc tensor(0.2083, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.3000, device='cuda:0')
self.adv_test_acc tensor(0.2083, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.3000, device='cuda:0')
self.adv_test_acc tensor(0.2083, device='cuda:0')
batch_size 120
x_adv torch.S

...

x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.4000, device='cuda:0')
self.adv_test_acc tensor(0.0250, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.4000, device='cuda:0')
self.adv_test_acc tensor(0.0250, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.4000, device='cuda:0')
self.adv_test_acc tensor(0.0250, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.4000, device='cuda:0')
self.adv_test_acc tensor(0.0250, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.4000, device='cuda:0')
self.adv_test_acc tensor(0.0250, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.4000, device='cuda:0')
self.adv_test_acc tensor(0.0250, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.4000, device='cuda:0')
self.adv_test_acc tensor(0.0250, device='cuda:0')
batch_size 120
x_adv torch.S

x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.3750, device='cuda:0')
self.adv_test_acc tensor(0., device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.3750, device='cuda:0')
self.adv_test_acc tensor(0., device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.3750, device='cuda:0')
self.adv_test_acc tensor(0., device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.3750, device='cuda:0')
self.adv_test_acc tensor(0., device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.3750, device='cuda:0')
self.adv_test_acc tensor(0., device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.3750, device='cuda:0')
self.adv_test_acc tensor(0., device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.3750, device='cuda:0')
self.adv_test_acc tensor(0., device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_a

orig_acc tensor(0.3583, device='cuda:0')
self.adv_test_acc tensor(0.2833, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.3583, device='cuda:0')
self.adv_test_acc tensor(0.2833, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.3583, device='cuda:0')
self.adv_test_acc tensor(0.2833, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.3583, device='cuda:0')
self.adv_test_acc tensor(0.2833, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.3583, device='cuda:0')
self.adv_test_acc tensor(0.2833, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.3583, device='cuda:0')
self.adv_test_acc tensor(0.2833, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.3583, device='cuda:0')
self.adv_test_acc tensor(0.2833, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tens

x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.3917, device='cuda:0')
self.adv_test_acc tensor(0.2500, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.3917, device='cuda:0')
self.adv_test_acc tensor(0.2500, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.3917, device='cuda:0')
self.adv_test_acc tensor(0.2500, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.3917, device='cuda:0')
self.adv_test_acc tensor(0.2500, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.3917, device='cuda:0')
self.adv_test_acc tensor(0.2500, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.3917, device='cuda:0')
self.adv_test_acc tensor(0.2500, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.3917, device='cuda:0')
self.adv_test_acc tensor(0.2500, device='cuda:0')
batch_size 120
x_adv torch.S

x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.3917, device='cuda:0')
self.adv_test_acc tensor(0.0167, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.3917, device='cuda:0')
self.adv_test_acc tensor(0.0167, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.3917, device='cuda:0')
self.adv_test_acc tensor(0.0167, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.3917, device='cuda:0')
self.adv_test_acc tensor(0.0167, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.3917, device='cuda:0')
self.adv_test_acc tensor(0.0167, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.3917, device='cuda:0')
self.adv_test_acc tensor(0.0167, device='cuda:0')
batch_size 120
x_adv torch.Size([120, 3, 32, 32])
orig_acc tensor(0.3917, device='cuda:0')
self.adv_test_acc tensor(0.0167, device='cuda:0')
batch_size 120
x_adv torch.S

In [10]:
# Aggregate Results Across clients 
metrics = ['orig_acc_transfers','orig_similarities','adv_acc_transfers','adv_similarities_target',
           'adv_similarities_untarget','adv_target','adv_miss'] #,'metric_alignment']

orig_acc = np.zeros([len(victim_idxs),len(victim_idxs)]) 
orig_sim = np.zeros([len(victim_idxs),len(victim_idxs)]) 
adv_acc = np.zeros([len(victim_idxs),len(victim_idxs)]) 
adv_sim_target = np.zeros([len(victim_idxs),len(victim_idxs)]) 
adv_sim_untarget = np.zeros([len(victim_idxs),len(victim_idxs)]) 
adv_target = np.zeros([len(victim_idxs),len(victim_idxs)]) # fraction of adv examples classified as target class
adv_miss = np.zeros([len(victim_idxs),len(victim_idxs)]) # fraction of adv examples classified correctly

for adv_idx in range(len(victim_idxs)):
    for victim in range(len(victim_idxs)):
        orig_acc[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[0]][victim_idxs[victim]].data.tolist()
        orig_sim[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[1]][victim_idxs[victim]].data.tolist()
        adv_acc[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[2]][victim_idxs[victim]].data.tolist()
        adv_sim_target[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[3]][victim_idxs[victim]].data.tolist()
        adv_sim_untarget[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[4]][victim_idxs[victim]].data.tolist()
        adv_target[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[5]][victim_idxs[victim]].data.tolist()
        adv_miss[adv_idx,victim] = logs_adv[victim_idxs[adv_idx]][metrics[6]][victim_idxs[victim]].data.tolist()

In [11]:
# Check Results
print(orig_acc[:10,:10])

for i in range(0,40):
    for j in range(0,40):
        print(adv_acc[i,j])

[[0.23 0.23 0.23 0.23 0.23 0.23 0.23 0.23 0.23 0.23]
 [0.23 0.23 0.23 0.23 0.23 0.23 0.23 0.23 0.23 0.23]
 [0.22 0.22 0.22 0.22 0.22 0.22 0.22 0.22 0.22 0.22]
 [0.30 0.30 0.30 0.30 0.30 0.30 0.30 0.30 0.30 0.30]
 [0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25]
 [0.27 0.27 0.27 0.27 0.27 0.27 0.27 0.27 0.27 0.27]
 [0.35 0.35 0.35 0.35 0.35 0.35 0.35 0.35 0.35 0.35]
 [0.52 0.52 0.52 0.52 0.52 0.52 0.52 0.52 0.52 0.52]
 [0.43 0.43 0.43 0.43 0.43 0.43 0.43 0.43 0.43 0.43]
 [0.33 0.33 0.33 0.33 0.33 0.33 0.33 0.33 0.33 0.33]]
0.13333334028720856
0.13333334028720856
0.13333334028720856
0.13333334028720856
0.13333334028720856
0.13333334028720856
0.13333334028720856
0.13333334028720856
0.13333334028720856
0.13333334028720856
0.13333334028720856
0.13333334028720856
0.13333334028720856
0.13333334028720856
0.13333334028720856
0.13333334028720856
0.13333334028720856
0.13333334028720856
0.13333334028720856
0.13333334028720856
0.13333334028720856
0.13333334028720856
0.13333334028720856
0.1333333

In [12]:
print(np.mean(adv_acc))

0.2135038012638688


In [13]:
# tmp = orig_acc
# tmp = np.where(np.isnan(orig_acc),-1,orig_acc)
# ct = (np.where(np.isnan(orig_acc),-1,orig_acc)==-1).sum()
# ct1 = (np.where(np.isnan(orig_acc),-1,orig_acc)!=-1).sum()
# s = np.sum()

In [14]:
print(np.mean(orig_acc))
print(np.mean(adv_acc))
print(np.mean(adv_target))

0.31645835041999815
0.2135038012638688
0.1441669123247266


In [15]:
np.save('results/FedAvg/{setting}_orig_acc_transfers'.format(setting=setting), orig_acc)
np.save('results/FedAvg/{setting}_orig_similarities'.format(setting=setting), orig_sim)
np.save('results/FedAvg/{setting}_adv_acc_transfers'.format(setting=setting), adv_acc)
np.save('results/FedAvg/{setting}_adv_similarities_target'.format(setting=setting), adv_sim_target)
np.save('results/FedAvg/{setting}_adv_similarities_untarget'.format(setting=setting), adv_sim_untarget)
np.save('results/FedAvg/{setting}_adv_target'.format(setting=setting), adv_target)
np.save('results/FedAvg/{setting}_adv_miss'.format(setting=setting), adv_miss)

In [16]:
'results/benign/{setting}_orig_acc_transfers'.format(setting=setting)

'results/benign/FedAvg_orig_acc_transfers'

In [17]:
setting

'FedAvg'

In [18]:
print('adv_acc:', avg_nondiag(adv_acc, skipNan=True))

adv_acc: 0.2135038012638688


In [19]:
print('adv_target:', avg_nondiag(adv_target, skipNan=True))
print('adv_miss:', avg_nondiag(adv_miss, skipNan=True))
print('orig_acc:', np.mean(np.diagonal(orig_acc)))

adv_target: 0.1441669123247266
adv_miss: 0.016458334354683756
orig_acc: 0.31645835041999815


In [20]:
print('orig_acc:', np.mean(np.diagonal(orig_acc)))

orig_acc: 0.31645835041999815


In [21]:
print(orig_acc[:10,:10])

[[0.23 0.23 0.23 0.23 0.23 0.23 0.23 0.23 0.23 0.23]
 [0.23 0.23 0.23 0.23 0.23 0.23 0.23 0.23 0.23 0.23]
 [0.22 0.22 0.22 0.22 0.22 0.22 0.22 0.22 0.22 0.22]
 [0.30 0.30 0.30 0.30 0.30 0.30 0.30 0.30 0.30 0.30]
 [0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25 0.25]
 [0.27 0.27 0.27 0.27 0.27 0.27 0.27 0.27 0.27 0.27]
 [0.35 0.35 0.35 0.35 0.35 0.35 0.35 0.35 0.35 0.35]
 [0.52 0.52 0.52 0.52 0.52 0.52 0.52 0.52 0.52 0.52]
 [0.43 0.43 0.43 0.43 0.43 0.43 0.43 0.43 0.43 0.43]
 [0.33 0.33 0.33 0.33 0.33 0.33 0.33 0.33 0.33 0.33]]
